In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 20.7 MB/s eta 0:00:00


In [3]:
from catboost import CatBoostClassifier

import pickle

step_1 = CatBoostClassifier().load_model(data_path + 'data/step1_model.dat')
with open(data_path + 'data/step2_model.dat', 'rb') as fp:
    step_2 = pickle.load(fp)
with open(data_path + 'data/step3_model.dat', 'rb') as fp:
    step_3 = pickle.load(fp)
    scaler = pickle.load(fp)
with open(data_path + 'data/step4_model.dat', 'rb') as fp:
    step_4 = pickle.load(fp)

In [4]:
x = pd.read_parquet(data_path + 'processed/train_all.parquet')
y = pd.read_csv(data_path + 'data/one_hot_encoding.csv')

In [5]:
x.drop(['ID', '기준년월'], axis=1, inplace=True)

In [6]:
x_scaled = scaler.transform(x)

In [7]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 310 entries, 이용금액_R3M_신용체크 to 변동률_RV평잔
dtypes: float64(32), int16(75), int32(68), int8(135)
memory usage: 1.8 GB


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [25]:
sub = pd.read_csv(data_path + 'processed/segment.csv')

In [26]:
sub

,ID,Segment
0,TRAIN_000000,D
1,TRAIN_000001,E
2,TRAIN_000002,C
3,TRAIN_000003,D
4,TRAIN_000004,E
...,...,...
2399995,TRAIN_399995,E
2399996,TRAIN_399996,D
2399997,TRAIN_399997,C
2399998,TRAIN_399998,E


In [39]:
threshold = 0.5
y_proba = pd.DataFrame()

In [42]:
proba_1 = step_1.predict_proba(X_train)[:,1]

In [45]:
y_proba['E'] = (proba_1 > threshold).astype(int)

In [47]:
X_train2 = X_train[y_proba['E'] == 0]
proba_2 = step_2.predict_proba(X_train2)[:,1]

In [48]:
col = pd.Series(0, index=y_proba.index)

In [49]:
col.loc[y_proba.index[y_proba['E'] == 0]] = (proba_2 > threshold).astype(int)

In [54]:
y_proba['D'] = col
X_train3 = X_train2[proba_2 <= threshold]
proba_3 = step_3.predict_proba(X_train3)[:,1]

In [55]:
col = pd.Series(0, index=y_proba.index)

In [65]:
mask = (y_proba['E'] == 0) & (y_proba['D'] == 0)
col.loc[mask] = (proba_3 > threshold).astype(int)

In [66]:
y_proba['C'] = col
X_train4 = X_train3[proba_3 <= threshold]

In [67]:
proba_4 = step_4.predict_proba(X_train4)[:,1]

In [68]:
col1 = pd.Series(0, index=y_proba.index)
col2 = pd.Series(0, index=y_proba.index)

In [69]:
mask = mask & (y_proba['C'] == 0)
col1.loc[mask] = (proba_4 > threshold).astype(int)
col2.loc[mask] = 1 - (proba_4 > threshold).astype(int)

In [70]:
col1.sum()

np.int64(5)

In [71]:
col2.sum()

np.int64(218)

In [72]:
y_proba['B'] = col1
y_proba['A'] = col2

In [73]:
y_proba

,A,E,D,C,B
0,0,1,0,0,0
1,0,0,1,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
1919995,0,1,0,0,0
1919996,0,1,0,0,0
1919997,0,1,0,0,0
1919998,0,1,0,0,0


In [79]:
y_decoded = y_proba.idxmax(axis=1)
y_train_decoded = y_train.idxmax(axis=1)

In [80]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_decoded, y_decoded)
cm

array([[    192,       0,      60,      35,     484],
       [     16,       5,      25,       0,      53],
       [      4,       0,   13059,    4432,   84563],
       [      0,       0,    7052,    7319,  264902],
       [      6,       0,    1764,    3579, 1532450]])

In [81]:
from sklearn.metrics import f1_score

f1 = f1_score(y_train_decoded, y_decoded, average='macro')
f1

0.3281616832505086